In [1]:
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import seaborn as sns
import numpy as np

%matplotlib inline

In [2]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier, RandomForestClassifier

In [3]:
hm_days = 7


def process_data_for_labels(ticker):
    df = pd.read_csv('sp500_joined_adjcloses.csv', index_col=0)
    tickers = df.columns.values.tolist()
    df.fillna(0, inplace=True)

    for i in range(1, hm_days+1):
        df['{}_{}d'.format(ticker, i)] = (
            df[ticker].shift(-i) - df[ticker]) / df[ticker]

    df.fillna(0, inplace=True)
    return tickers, df

In [4]:
# process_data_for_labels('XOM')

In [5]:
def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.025
    for col in cols:
        if col > requirement:
            return 1
        elif col < -requirement:
            return -1
    return 0

In [6]:
def extract_featuresets(ticker):
    tickers, df = process_data_for_labels(ticker)

    df['{}_target'.format(ticker)] = list(map(buy_sell_hold, *[df['{}_{}d'.format(ticker, i)]
                                                               for i in range(1, hm_days+1)]))

#                                               df['{}_1d'.format(ticker)],
#                                               df['{}_2d'.format(ticker)],
#                                               df['{}_3d'.format(ticker)],
#                                               df['{}_4d'.format(ticker)],
#                                               df['{}_5d'.format(ticker)],
#                                               df['{}_6d'.format(ticker)],
#                                               df['{}_7d'.format(ticker)]))

    vals = df['{}_target'.format(ticker)].values.tolist()
    str_vals = [str(i) for i in vals]
    print('Data spread:', Counter(str_vals))

    df.fillna(0, inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)

    df_vals = df[[ticker for ticker in tickers]].pct_change()
    df_vals = df_vals.replace([np.inf, -np.inf], 0)
    df_vals.fillna(0, inplace=True)

    X, y = df_vals.values, df['{}_target'.format(ticker)].values
    return X, y, df

In [7]:
# X, y, df = extract_featuresets('XOM')

In [8]:
def do_ml(ticker):
    X, y, df = extract_featuresets(ticker)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

#     clf = KNeighborsClassifier()
    clf = VotingClassifier(
        [('lsvc', SVC()), ('knn', KNeighborsClassifier()), ('rfc', RandomForestClassifier())])

    clf.fit(X_train, y_train)

    confidence = clf.score(X_test, y_test)
    print('Accuracy', confidence)

    y_predict = clf.predict(X_test)
    print('Predicted spread:', Counter(y_predict),'True spread:', Counter(y_test))

    return confidence

In [9]:
do_ml('BAC')

Data spread: Counter({'1': 1587, '-1': 1476, '0': 1216})
Accuracy 0.44485981308411215
Predicted spread: Counter({-1: 429, 1: 347, 0: 294}) True spread: Counter({1: 393, -1: 372, 0: 305})


0.44485981308411215